In [5]:
import os
import numpy as np
import pandas as pd
import scipy.io as spio
from openpyxl import Workbook

In [30]:
# Specify the directory where the MATLAB data files are located
data_dir = '/Users/linagladutyte/Documents/LABES/RAMYBE/FOOF/params_all_subjects/'

In [31]:
# Get a list of file names in the directory
file_list = os.listdir(data_dir)

In [34]:
# Create empty dataset 
double_peaks_per_subject = pd.DataFrame(columns=['subject', 'double_peak_count'])
no_peaks_per_subject = pd.DataFrame(columns=['subject', 'no_peak_count'])

# Create separate lists for subject IDs and double peak counts
subject_ids = []
double_peak_counts = []
peak_number_rsq_ch = []
no_peak_number_rsq_ch = []
no_peak_counts = []

# Iterate over each MATLAB data file
for i, file_name in enumerate(file_list):
    # Load th MATLAB data file
    mat_data = spio.loadmat(os.path.join(data_dir, file_name))
    peak_params = mat_data['peak_params']  # center frequency(CF); power (PW); band width (BW)
    r_squared = mat_data['r_squared']

    # Get the channel names and subject name from the file name
    channels = peak_params.dtype.names
    name = file_name.split('_parameters')[0]
    
    # Create an empty list to store the number of peaks
    double_peaks = []
    no_peaks = []

    # Get the channel names and subject name from the file name
    channels = peak_params.dtype.names
    name = file_name.split('_parameters')[0]

    # Iterate over each channel in the data file
    for ch in range(len(channels)):
        rsq = r_squared[0][0][ch][0][0]
        peak = peak_params[0][0][ch]

        # Extract the CF values for the current channel
        cf_values = peak[:, 0]

        # Count alpha peaks (the number of CF values within the specified range)
        count = np.sum((cf_values >= 8) & (cf_values <= 12))

        if count >= 2:
            double_peaks.append([name, rsq, count, ch])
        if count < 1:
            no_peaks.append([name, rsq, count, ch])

    # Check if peaks is not empty
    if double_peaks:
        peak_number_rsq_ch.append(double_peaks)

    # Check if no_peaks is not empty before appending
    if no_peaks:
        no_peak_number_rsq_ch.append(no_peaks)

    # Append subject ID and double peak count to respective lists
    subject_ids.append(name)
    double_peak_counts.append(len(double_peaks))

    # Append subject ID and no peak count to respective lists
    no_peak_counts.append(len(no_peaks))

# Update the double_peaks_per_subject DataFrame with subject IDs and double peak counts
double_peaks_per_subject['subject'] = subject_ids
double_peaks_per_subject['double_peak_count'] = double_peak_counts

# Update the no_peaks_per_subject DataFrame with subject IDs and no peak counts
no_peaks_per_subject['subject'] = subject_ids
no_peaks_per_subject['no_peak_count'] = no_peak_counts

In [41]:
data_frames = [
    ('double_peaks_per_subject.xlsx', double_peaks_per_subject),
    ('no_alpha_per_subject', no_peaks_per_subject)
]

with pd.ExcelWriter('alpha_peaks_count.xlsx') as writer:
    for sheet_name, data_frame in data_frames:
        data_frame.to_excel(writer, sheet_name=sheet_name, index=False)